In [1]:
print(__doc__)

# Common imports
import numpy as np
import numpy.random as rnd
import os
import pandas as pd

# File specific imports
import datetime
from dateutil.relativedelta import relativedelta

# To make this notebook's output stable across runs
rnd.seed(42)

# Show all columns when displaying dataframes
pd.set_option('display.max_columns', None)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 10
plt.rcParams['ytick.labelsize'] = 10

Automatically created module for IPython interactive environment


In [2]:
csv_path = '/media/bigdatabuffalo/drive/MIMIC3/CSV/ADMISSIONS.csv'
cols_to_keep = ['SUBJECT_ID', 'HADM_ID', 'ADMITTIME', 'DISCHTIME', 'ADMISSION_LOCATION', 'ETHNICITY', 'DIAGNOSIS']
admissions = pd.read_csv(csv_path, skipinitialspace = True, usecols = cols_to_keep)

csv_path = '/media/bigdatabuffalo/drive/MIMIC3/CSV/PATIENTS.csv'
cols_to_keep = ['SUBJECT_ID', 'GENDER', 'DOB', 'EXPIRE_FLAG']
patients = pd.read_csv(csv_path, skipinitialspace = True, usecols = cols_to_keep)

csv_path = '/media/bigdatabuffalo/drive/MIMIC3/CSV/INPUTEVENTS_CV.csv'
cols_to_keep = ['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'ORDERID', 'CHARTTIME', 'ITEMID', 'AMOUNT', 'AMOUNTUOM', 'ORIGINALAMOUNT', 'ORIGINALAMOUNTUOM', 'ORIGINALRATE', 'ORIGINALRATEUOM'] # 'RATE', 'RATEUOM', 'STOPPED' are full of NaN
inputevents_cv = pd.read_csv(csv_path, skipinitialspace = True, usecols = cols_to_keep)

csv_path = '/media/bigdatabuffalo/drive/MIMIC3/CSV/INPUTEVENTS_MV.csv'
cols_to_keep = ['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'ORDERID', 'STARTTIME', 'ENDTIME', 'ITEMID', 'AMOUNT', 'AMOUNTUOM', 'RATE', 'RATEUOM', 'ORDERCATEGORYNAME', 'ORDERCATEGORYDESCRIPTION']
inputevents_mv = pd.read_csv(csv_path, skipinitialspace = True, usecols = cols_to_keep)

csv_path = '/media/bigdatabuffalo/drive/MIMIC3/CSV/LABEVENTS.csv'
cols_to_keep = ['SUBJECT_ID', 'HADM_ID', 'ITEMID', 'CHARTTIME', 'VALUE', 'VALUENUM', 'VALUEUOM', 'FLAG'] # 'HADM_ID' is full of NaN
labevents = pd.read_csv(csv_path, skipinitialspace = True, usecols = cols_to_keep)

csv_path = '/media/bigdatabuffalo/drive/MIMIC3/CSV/D_ITEMS.csv'
cols_to_keep = ['ITEMID', 'LABEL'] #'CATEGORY', 'UNITNAME', 'PARAM_TYPE' are full of NaN
d_items = pd.read_csv(csv_path, skipinitialspace = True, usecols = cols_to_keep)

csv_path = '/media/bigdatabuffalo/drive/MIMIC3/CSV/D_LABITEMS.csv'
cols_to_keep = ['ITEMID', 'LABEL', 'FLUID', 'CATEGORY']
d_labitems = pd.read_csv(csv_path, skipinitialspace = True, usecols = cols_to_keep)

/usr/local/lib/python3.4/dist-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (7,17,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
admissions = admissions.applymap(str)
patients = patients.applymap(str)
inputevents_cv = inputevents_cv.applymap(str)
inputevents_mv  = inputevents_mv.applymap(str)
labevents  = labevents.applymap(str) 
d_items = d_items.applymap(str)
d_labitems = d_labitems.applymap(str)

# PATIENT INFORMATION

In [4]:
# Combine admissions and patients
patient_info = pd.merge(admissions, patients, on = ['SUBJECT_ID'])

# Change Ethnicity to White/Non-White
ethnicity = patient_info['ETHNICITY']

def get_ethnicity(value):
    if value == 'WHITE':
        return 'WHITE'
    else:
        return 'NOT WHITE'

list_ethnicity = [get_ethnicity(value) for value in ethnicity]

patient_info = patient_info.drop('ETHNICITY', axis = 1)
df_ethnicity = pd.DataFrame({'ETHNICITY' : list_ethnicity})
patient_info = pd.concat([patient_info, df_ethnicity], axis=1)

# Add ages
DoB = patient_info['DOB']
admDate = patient_info['ADMITTIME']

def get_age (birth, current):
    #replace dashes and colons with spaces to make breaking the string up easier
    birth = birth.replace('-', ' ')
    birth = birth.replace(':', ' ')
    current = current.replace('-', ' ')
    current = current.replace(':', ' ')

    #split into a string outputting [year, month, day, hour, minutes, seconds]
    birth = birth.split(' ')
    current = current.split(' ')

    #convert to integers
    birth = [int(i) for i in birth]
    current = [int(i) for i in current]
    
    #get age and return it
    birth = datetime.datetime(birth[0], birth[1], birth[2], birth[3], birth[4], birth[5])
    current = datetime.datetime(current[0], current[1], current[2], current[3], current[4], current[5])
    age = relativedelta(current, birth).years
    
    #adjust for 89+ category
    if age == 300:
        age = 89
    
    return age

def get_age_category(age):
    if age > 88:
        return ('89+')
    elif age > 60:
        return('Elderly')
    elif age > 50:
        return('50s')
    elif age > 40:
        return('40s')
    elif age > 30:
        return('30s')
    elif age > 17:
        return('20s')
    else:
        return('Child')

ages = []
age_categories = []

for i in range(len(DoB)):
    ages.append(get_age(DoB[i], admDate[i]))

for age in ages:
    age_categories.append(get_age_category(age))

ages_df = pd.DataFrame({'AGE' : ages})
ages_cat_df = pd.DataFrame({'AGE_CAT' : age_categories})
patient_info = pd.concat([patient_info, ages_df], axis=1)
patient_info = pd.concat([patient_info, ages_cat_df], axis=1)

# Add LOS
admDate = patient_info['ADMITTIME']
dischDate = patient_info['DISCHTIME']

def get_los (admit, disch):
    #replace dashes and colons with spaces to make breaking the string up easier
    admit = admit.replace('-', ' ')
    admit = admit.replace(':', ' ')
    disch = disch.replace('-', ' ')
    disch = disch.replace(':', ' ')

    #split into a string outputting [year, month, day, hour, minutes, seconds]
    admit = admit.split(' ')
    disch = disch.split(' ')

    #convert to integers
    admit = [int(i) for i in admit]
    disch = [int(i) for i in disch]
    
    #get los and return it
    admit = datetime.datetime(admit[0], admit[1], admit[2], admit[3], admit[4], admit[5])
    disch = datetime.datetime(disch[0], disch[1], disch[2], disch[3], disch[4], disch[5])
    los = (disch-admit).total_seconds()/3600
    los = round((los/24), 1)

    return los

los = []

for i in range(len(admDate)):
    los.append(get_los(admDate[i], dischDate[i]))
    
los_df = pd.DataFrame({'LOS' : los})
patient_info = pd.concat([patient_info, los_df], axis = 1)

# Get Day/Night (day is between 8 am : 8 pm)
admTime = patient_info['ADMITTIME']

def get_day_night (value):
    #replace dashes and colons with spaces to make breaking the string up easier
    value = value.replace('-', ' ')
    value = value.replace(':', ' ')
    
    #split into a string outputting [year, month, day, hour, minutes, seconds]
    date = value.split(' ')
    
    #leave hour/min/sec, convert to integers
    hour = date[3]
    int_hour = int(hour)
    
    #get day/night and return it
    if 7 < int_hour < 21:
        return 'DAY'
    else:
        return 'NIGHT'

day_night = [get_day_night(value) for value in admTime]

day_night_df = pd.DataFrame({'DAY_NIGHT' : day_night})
patient_info = pd.concat([patient_info, day_night_df], axis=1)

# Get admission day
admTime = patient_info['ADMITTIME']

def get_day_of_week (value):
    date = []
    int_date = []
    day_of_week = []
    
    #replace dashes and colons with spaces to make breaking the string up easier
    value = value.replace('-', ' ')
    value = value.replace(':', ' ')
    
    #split into a string outputting [year, month, day, hour, minutes, seconds]
    date = value.split(' ')
    
    #delete hour/min/sec, convert to integers
    del date[3:6]
    int_date = [int(i) for i in date]
    
    #get day of week and return it
    day_of_week = datetime.date(int_date[0], int_date[1], int_date[2]).weekday()
    
    return day_of_week


admTime_weekday = [get_day_of_week(value) for value in admTime]

weekday_data_df = pd.DataFrame({'ADM_DAY' : admTime_weekday})
patient_info = pd.concat([patient_info, weekday_data_df], axis=1)

# Clean up columns
patient_times_info = patient_info[['SUBJECT_ID', 'ADMITTIME', 'DISCHTIME', 'DOB']].copy()
patient_times_info = patient_times_info.drop_duplicates(subset = 'SUBJECT_ID', keep = 'first')

patient_info = patient_info.drop(['ADMITTIME', 'DISCHTIME', 'DOB'], axis = 1)

LABEVENTS

In [5]:
# Convert labevents[FLAG] : NaN to 0, abnomal to 1
labevents['FLAG'] = labevents['FLAG'].replace('nan', 0)
labevents['FLAG'] = labevents['FLAG'].replace('abnormal', 1)

# Merge d_labitems onto labevents 
labevents = pd.merge(labevents, d_labitems, on = 'ITEMID')

INPUTEVENTS (CV AND MV)

In [6]:
# Merge d_items onto inputevents_cv, and inputevents_mv separately
inputevents_cv = pd.merge(inputevents_cv, d_items, on = 'ITEMID')
inputevents_mv = pd.merge(inputevents_mv, d_items, on = 'ITEMID')

# Heparin Data Organization

In [7]:
patient_info.head()

,SUBJECT_ID,HADM_ID,ADMISSION_LOCATION,DIAGNOSIS,GENDER,EXPIRE_FLAG,ETHNICITY,AGE,AGE_CAT,LOS,DAY_NIGHT,ADM_DAY
0,22,165315,EMERGENCY ROOM ADMIT,BENZODIAZEPINE OVERDOSE,F,0,WHITE,64,Elderly,1.1,DAY,5
1,23,152223,PHYS REFERRAL/NORMAL DELI,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,M,0,WHITE,71,Elderly,5.5,NIGHT,0
2,23,124321,TRANSFER FROM HOSP/EXTRAM,BRAIN MASS,M,0,WHITE,75,Elderly,6.8,DAY,1
3,24,161859,TRANSFER FROM HOSP/EXTRAM,INTERIOR MYOCARDIAL INFARCTION,M,0,WHITE,39,30s,2.9,DAY,5
4,25,129635,EMERGENCY ROOM ADMIT,ACUTE CORONARY SYNDROME,M,0,WHITE,58,50s,3.5,NIGHT,6


1\. UNIQUE ADULTS

In [8]:
# Get only first ICU stay for each patient
patient_info = patient_info.drop_duplicates(subset = 'SUBJECT_ID', keep = 'first')

# Find all Heparin patients
word_list = ['Heparin']
heparin_cv = inputevents_cv.loc[inputevents_cv['LABEL'].isin(word_list)]
heparin_mv = inputevents_mv[inputevents_mv['LABEL'].str.contains('|'.join(word_list))]

# Drop Prophylaxis from heparin_mv
heparin_mv = heparin_mv[heparin_mv['LABEL'] != 'Heparin Sodium (Prophylaxis)']

# Count total patients recieving UFH
gp_cv = heparin_cv['SUBJECT_ID'].unique().tolist()
gp_mv = heparin_mv['SUBJECT_ID'].unique().tolist()
patient_count = len(gp_cv) + len(gp_mv)
print('Total PT recieving UFH: ', patient_count)

Total PT recieving UFH:  8095


In [9]:
# Update patient_info to heparin_patients
patient_list = gp_cv + gp_mv

heparin_patients = pd.DataFrame({'SUBJECT_ID' : patient_list})
heparin_patients = heparin_patients.drop_duplicates(subset = 'SUBJECT_ID', keep = 'first')
heparin_patients = pd.merge(heparin_patients, patient_info, on = 'SUBJECT_ID')
heparin_patients.shape

(7974, 12)

2\. aPTT MEASUREMENT

In [10]:
# Find all aPTT measurements
word_list = ['PTT']
aPTT = labevents[labevents['LABEL'].str.contains('|'.join(word_list))]
aPTT = aPTT.reset_index(drop = True)
aPTT['HADM_ID'] = aPTT['HADM_ID'].str.replace('.0', '')

# Remove repeat HADM_ID
word_list = heparin_patients['HADM_ID'].tolist()
aPTT = aPTT.loc[aPTT['HADM_ID'].isin(word_list)]

# Create Groups
gp_ptt = aPTT['SUBJECT_ID'].unique().tolist()
patient_count = len(gp_ptt)

print('Total PT with aPTT tests: ', patient_count)

Total PT with aPTT tests:  4195


In [11]:
# Update heparin_patients
patient_list_ptt = gp_ptt
aPTT_patients = pd.DataFrame({'SUBJECT_ID' : patient_list_ptt})
heparin_patients = pd.merge(heparin_patients, aPTT_patients, on = 'SUBJECT_ID')
heparin_patients.shape

(4195, 12)

In [12]:
ptts = aPTT['VALUE']

def get_aPTT(ptt):
    if ptt == '>150':
        return ('150.0')
    elif ptt == '>150.0':
        return ('150.0')
    elif ptt == '> 150':
        return ('150.0')
    elif len(ptt) > 5:
        return (None)
    elif ptt == 'ERROR':
        return (None)
    ptt = ptt.replace('..', '.')
    return(ptt)

def get_aPTT_category(ptt):
    if ptt == None:
        return (None)
    ptt = float(ptt)
    if ptt > 100:
        return ('SUPRA-TH')
    elif ptt < 60:
        return('SUB-TH')
    else:
        return('TH')

ptts_new = []
ptt_categories = []

for ptt in ptts:
    ptts_new.append(get_aPTT(ptt))

for ptt in ptts_new:
    ptt_categories.append(get_aPTT_category(ptt))

ptt_df = pd.DataFrame({'aPTT' : ptts_new})
ptt_cat_df = pd.DataFrame({'aPTT_CAT' : ptt_categories})

aPTT = aPTT.reset_index(drop = True)
aPTT = pd.concat([aPTT, ptt_df], axis = 1)
aPTT = pd.concat([aPTT, ptt_cat_df], axis = 1)
aPTT = aPTT[['SUBJECT_ID', 'HADM_ID', 'ITEMID', 'CHARTTIME', 'aPTT', 'VALUEUOM', 'LABEL', 'aPTT_CAT', 'FLUID', 'CATEGORY']].copy()

3\. NON TRANSFERS

In [13]:
heparin_patients = heparin_patients[heparin_patients.ADMISSION_LOCATION != 'TRANSFER FROM SKILLED NUR']
heparin_patients = heparin_patients[heparin_patients.ADMISSION_LOCATION != 'TRANSFER FROM HOSP/EXTRAM']
heparin_patients = heparin_patients[heparin_patients.ADMISSION_LOCATION != 'TRANSFER FROM OTHER HEALT']
heparin_patients = heparin_patients[heparin_patients.ADMISSION_LOCATION != '** INFO NOT AVAILABLE **']

heparin_patients.shape

(2914, 12)

4\. FEATURES

Dosage Start Time

In [14]:
# Update heparin dosage dataframes
word_list = heparin_patients['HADM_ID'].tolist()
heparin_cv['HADM_ID'] = heparin_cv['HADM_ID'].str.replace('.0', '')
heparin_cv = heparin_cv.loc[heparin_cv['HADM_ID'].isin(word_list)]
heparin_mv = heparin_mv.loc[heparin_mv['HADM_ID'].isin(word_list)]

# Extract IDs and Start Times
dose_cv = heparin_cv[['SUBJECT_ID', 'CHARTTIME']].copy()
dose_cv = dose_cv.rename(columns = {'CHARTTIME': 'DOSE_TIME'})

dose_mv = heparin_mv[['SUBJECT_ID', 'STARTTIME']].copy()
dose_mv = dose_mv.rename(columns = {'STARTTIME': 'DOSE_TIME'})

# Combine and take First Dose
dose_info = pd.concat([dose_cv, dose_mv])
first_dose = dose_info.drop_duplicates(subset = 'SUBJECT_ID', keep = 'first')

#Update patient numbers
x = heparin_cv['SUBJECT_ID'].unique().tolist()
y = heparin_mv['SUBJECT_ID'].unique().tolist()
z = x+y
z_df = pd.DataFrame({'SUBJECT_ID' : z})
z_df = z_df.drop_duplicates(subset = 'SUBJECT_ID', keep = 'first')
heparin_patients = pd.merge(heparin_patients, z_df, on = 'SUBJECT_ID')
print(heparin_patients.shape)
print ('Total patients with first dose: ', len(z))

/usr/local/lib/python3.4/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


(2440, 12)
Total patients with first dose:  2440


MEASUREMENT TIME (dose to aPTT)

In [15]:
aPTT = aPTT.rename(columns = {'CHARTTIME': 'aPTT_TIME'})
measure_time = aPTT[['SUBJECT_ID', 'aPTT_TIME']].copy()
measure_time = pd.merge(measure_time, first_dose, on = 'SUBJECT_ID')

doseDate = measure_time['DOSE_TIME']
measureDate = measure_time['aPTT_TIME']

def get_length (first, second):
    #replace dashes and colons with spaces to make breaking the string up easier
    first = first.replace('-', ' ')
    first = first.replace(':', ' ')
    second = second.replace('-', ' ')
    second = second.replace(':', ' ')

    #split into a string outputting [year, month, day, hour, minutes, seconds]
    first = first.split(' ')
    second = second.split(' ')

    #convert to integers
    first = [int(i) for i in first]
    second = [int(i) for i in second]
    
    #get los and return it
    first = datetime.datetime(first[0], first[1], first[2], first[3], first[4], first[5])
    second = datetime.datetime(second[0], second[1], second[2], second[3], second[4], second[5])
    length = (second-first).total_seconds()/3600
    length = round(length, 1)

    return length

measure_list = []

for i in range(len(doseDate)):
    measure_list.append(get_length(doseDate[i], measureDate[i]))
    
measure_df = pd.DataFrame({'MEASURE_TIME' : measure_list})
measure_time = pd.concat([measure_time, measure_df], axis = 1)
measure_time.head()

,SUBJECT_ID,aPTT_TIME,DOSE_TIME,MEASURE_TIME
0,3,2101-10-22 04:30:00,2101-10-23 08:00:00,-27.5
1,3,2101-10-22 12:45:00,2101-10-23 08:00:00,-19.2
2,3,2101-10-22 21:15:00,2101-10-23 08:00:00,-10.8
3,3,2101-10-23 03:45:00,2101-10-23 08:00:00,-4.2
4,3,2101-10-23 10:10:00,2101-10-23 08:00:00,2.2


In [16]:
aPTT_dosage = pd.merge(aPTT, measure_time, on = ['SUBJECT_ID', 'aPTT_TIME'])

# Save rows with values between 4 and 10 hours
measurements = aPTT_dosage['MEASURE_TIME']

def get_measure (time):
    if time < 4:
        return (None)
    if time > 10:
        return (None)
    else:
        return (time)

measurements_list = [get_measure(value) for value in measurements]

measurements_df = pd.DataFrame({'MEASUREMENT_TIME' : measurements_list})
aPTT_dosage = pd.concat([aPTT_dosage, measurements_df], axis = 1)
aPTT_dosage = aPTT_dosage.dropna()
aPTT_dosage = aPTT_dosage.drop('MEASUREMENT_TIME', axis = 1)

DOSAGE TIME (adm to dose)

In [17]:
adm = patient_times_info[['SUBJECT_ID', 'ADMITTIME']].copy()
dosage_info = pd.merge(adm, first_dose, on = 'SUBJECT_ID')

admDate = dosage_info['ADMITTIME']
doseDate = dosage_info['DOSE_TIME']

dosage_list = []

for i in range(len(admDate)):
    dosage_list.append(get_length(admDate[i], doseDate[i]))
    
dosage_df = pd.DataFrame({'DOSAGE_TIME' : dosage_list})
dosage_time = pd.concat([dosage_info, dosage_df], axis = 1)
dosage_time.head()

,SUBJECT_ID,ADMITTIME,DOSE_TIME,DOSAGE_TIME
0,25,2160-11-02 02:06:00,2160-11-03 10:00:00,31.9
1,107,2115-02-20 17:41:00,2115-02-20 20:00:00,2.3
2,130,2119-10-29 14:49:00,2119-10-30 23:00:00,32.2
3,154,2117-12-29 21:36:00,2117-12-29 22:00:00,0.4
4,111,2142-04-24 06:55:00,2142-04-24 21:00:00,14.1


INITIAL HEPARIN DRIP

In [40]:
# Label 'Heparin' is going to be the focus because it is already in Units
word_list = ['Heparin']
heparin_units_cv = heparin_cv[heparin_cv['LABEL'].isin(word_list)]

heparin_units_mv = heparin_mv.copy()

# Clean up
heparin_units_cv = heparin_units_cv[heparin_units_cv.ORIGINALRATE != 'nan']
heparin_units_cv = heparin_units_cv.drop_duplicates(subset = 'SUBJECT_ID', keep = 'first')
heparin_units_cv = heparin_units_cv[['SUBJECT_ID', 'CHARTTIME', 'ORIGINALRATE']].copy()
heparin_units_cv = heparin_units_cv.rename(columns = {'CHARTTIME': 'STARTTIME'})
heparin_units_cv = heparin_units_cv.rename(columns = {'ORIGINALRATE': 'RATE'})
heparin_units_cv.RATE = heparin_units_cv.RATE.astype(float)
heparin_units_cv.loc[:, 'RATE'] *= 100

heparin_units_mv = heparin_units_mv[heparin_units_mv.RATE != 'nan']
heparin_units_mv = heparin_units_mv.drop_duplicates(subset = 'SUBJECT_ID', keep = 'first')
heparin_units_mv = heparin_units_mv[['SUBJECT_ID', 'STARTTIME', 'RATE']].copy()

In [41]:
# Combine into one dataframe
heparin_units = pd.concat([heparin_units_cv, heparin_units_mv])
heparin_units['RATE'] = pd.to_numeric(heparin_units['RATE']).astype(int)

# Update heparin_patients
patient_list_units = list(heparin_units['SUBJECT_ID'].unique())
units_patients = pd.DataFrame({'SUBJECT_ID' : patient_list_units})
heparin_patients = pd.merge(units_patients, heparin_patients, on = 'SUBJECT_ID')
heparin_patients.shape

(1409, 12)

COMBINE

In [50]:
# Merge dosage, measure, and patient information
aPTT_heparin = pd.merge(dosage_time, aPTT_dosage, on = ['SUBJECT_ID', 'DOSE_TIME'])
aPTT_heparin = pd.merge(heparin_patients, aPTT_heparin, on = ['SUBJECT_ID', 'HADM_ID'])
aPTT_heparin = pd.merge(heparin_units, aPTT_heparin, on = ['SUBJECT_ID'])
aPTT_heparin = aPTT_heparin.drop(['SUBJECT_ID', 'HADM_ID', 'STARTTIME', 'DIAGNOSIS', 'ADMITTIME', 'DOSE_TIME', 'ITEMID', 'aPTT_TIME', 'VALUEUOM', 'LABEL', 'FLUID', 'CATEGORY'], axis = 1)
aPTT_heparin.head()

,RATE,ADMISSION_LOCATION,GENDER,EXPIRE_FLAG,ETHNICITY,AGE,AGE_CAT,LOS,DAY_NIGHT,ADM_DAY,DOSAGE_TIME,aPTT,aPTT_CAT,MEASURE_TIME
0,900,EMERGENCY ROOM ADMIT,M,1,WHITE,89,89+,14.9,DAY,2,191.0,74.1,TH,4.0
1,1000,EMERGENCY ROOM ADMIT,M,1,WHITE,83,Elderly,54.7,DAY,4,5.8,60.4,TH,4.0
2,900,EMERGENCY ROOM ADMIT,F,1,WHITE,64,Elderly,6.9,NIGHT,1,77.1,150,SUPRA-TH,5.8
3,800,EMERGENCY ROOM ADMIT,M,0,WHITE,44,40s,2.7,NIGHT,2,0.4,47.5,SUB-TH,5.0
4,1000,EMERGENCY ROOM ADMIT,F,1,WHITE,57,50s,4.8,DAY,4,14.8,150,SUPRA-TH,6.5


SPLIT WEEKDAY VS WEEKEND

In [ ]:
word_list = ['0', '1', '2', '3']
heparin_weekday = aPTT_heparin.loc[aPTT_heparin['ADM_DAY'].isin(word_list)]

word_list = ['4', '5', '6']
heparin_weekend = aPTT_heparin.loc[aPTT_heparin['ADM_DAY'].isin(word_list)]

5\. WEEKDAY CATEGORIES : Target = aPTT_CAT

data_manipulation = heparin_weekday[['ADMISSION_LOCATION', 'GENDER', 'ETHNICITY', 'AGE_CAT', 'DAY_NIGHT', 'EXPIRE_FLAG', 'aPTT_CAT']].copy()
data_manipulation['aPTT_CAT'] = data_manipulation['aPTT_CAT'].replace('SUB-TH', '1')
data_manipulation['aPTT_CAT'] = data_manipulation['aPTT_CAT'].replace('TH', '0')
data_manipulation['aPTT_CAT'] = data_manipulation['aPTT_CAT'].replace('SUPRA-TH', '0')

X_num_day = heparin_weekday[['AGE', 'LOS', 'DOSAGE_TIME', 'MEASURE_TIME']].copy()
X_num_day = X_num_day.applymap(int)

X_cat_day = data_manipulation[['ADMISSION_LOCATION', 'GENDER', 'ETHNICITY', 'EXPIRE_FLAG']].copy() # removed 'AGE_CAT', 'DAY_NIGHT',
X_cat_day['EXPIRE_FLAG'] = X_cat_day['EXPIRE_FLAG'].astype(int)
X_cat_day = pd.get_dummies(X_cat_day)
X_weekday = pd.concat([X_cat_day, X_num_day], axis = 1)

X = X_weekday.as_matrix()

y = data_manipulation[['aPTT_CAT']].copy()
y = y_cat_day.as_matrix()

column_titles = list(X_weekday)
value_map = []
for value in range(len(column_titles)):
    x = column_titles[value], value
    value_map.append(x)
value_map

plt.figure(1)
plt.clf()

X_indices = np.arange(X.shape[-1])
X_indices

# Plot the feature score map
from sklearn.feature_selection import SelectPercentile, f_classif

selector = SelectPercentile(f_classif, percentile=10)
selector.fit(X, y)
scores = -np.log10(selector.pvalues_)
scores /= scores.max()
plt.bar(X_indices - .45, scores, width=.2,
        label=r'Univariate score ($-Log(p_{value})$)', color='darkorange')

# Show feature score map
score_map = []
for x in range(len(scores)):
    x = column_titles[x], scores[x]
    score_map.append(x)
score_map_df = pd.DataFrame(score_map)
score_map_df = score_map_df.rename(columns = {0: 'FEATURE', 1 : 'SCORE'})
score_map_df

pvalue_map = []
for x in range(len(selector.pvalues_)):
    x = column_titles[x], selector.pvalues_[x]
    pvalue_map.append(x)
pvalue_df = pd.DataFrame(pvalue_map)
pvalue_df = pvalue_df.rename(columns = {0: 'FEATURE', 1 : 'P-VALUE'})
pvalue_df

#SVC
from sklearn import datasets, svm

clf = svm.SVC(kernel='linear')
clf.fit(X, y)

svm_weights = (clf.coef_ ** 2).sum(axis=0)
svm_weights /= svm_weights.max()

plt.bar(X_indices - .25, svm_weights, width=.2, label='SVM weight',
        color='navy')

clf_selected = svm.SVC(kernel='linear')
clf_selected.fit(selector.transform(X), y)

svm_weights_selected = (clf_selected.coef_ ** 2).sum(axis=0)
svm_weights_selected /= svm_weights_selected.max()

plt.bar(X_indices[selector.get_support()] - .05, svm_weights_selected,
        width=.2, label='SVM weights after selection', color='c')


plt.title("Comparing feature selection")
plt.xlabel('Feature number')
plt.yticks(())
plt.axis('tight')
plt.legend(loc='upper right')
plt.show()

# SVM-ANOVA
from sklearn import svm, datasets, feature_selection
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline

transform = feature_selection.SelectPercentile(feature_selection.f_classif)

clf = Pipeline([('anova', transform), ('svc', svm.SVC(C=1.0))])

y_list = y.tolist()
y_new = []
for i in range(len(y_list)):
    y_new.append(y_list[i][0])
y_array = np.array(y_new)
y_array.shape

score_means = list()
score_stds = list()
percentiles = (1, 3, 6, 10, 15, 20, 30, 40, 60, 80, 100)

for percentile in percentiles:
    clf.set_params(anova__percentile=percentile)
    # Compute cross-validation score using 1 CPU
    this_scores = cross_val_score(clf, X, y_array, n_jobs=1)
    score_means.append(this_scores.mean())
    score_stds.append(this_scores.std())

plt.errorbar(percentiles, score_means, np.array(score_stds))

plt.title('Performance of the SVM-Anova varying the percentile of features selected')
plt.xlabel('Percentile')
plt.ylabel('Prediction rate')

plt.axis('tight')
plt.show()

Multivariate Analysis

from sklearn.model_selection import train_test_split

# Split into train/test sets
train_X, test_X = train_test_split(X, test_size=0.2, random_state=42)
train_y, test_y = train_test_split(y, test_size=0.2, random_state=42)

y_list = test_y.tolist()
y_new = []
for i in range(len(y_list)):
    y_new.append(y_list[i][0])
test_y = np.array(y_new)

# Fit
from sklearn.linear_model import LogisticRegression
lr_mn = LogisticRegression(multi_class='multinomial', solver ='newton-cg')
lr_mn.fit(train_X, train_y)

# Predict
from sklearn import metrics, cross_validation
preds = cross_validation.cross_val_predict(lr_mn, test_X, test_y, cv=10)
print ('Prediction accuracy: ', metrics.accuracy_score(test_y, preds))
print (metrics.classification_report(test_y, preds)) 

# Reciever Operating Characteristics (ROC)
from itertools import cycle
from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from scipy import interp
random_state = np.random.RandomState(42)

# Convert to Binarize formatting
y = data_manipulation[['aPTT_CAT']].copy()
yy = y.applymap(int).as_matrix()
y = label_binarize(yy, classes=[0, 1])
n_classes = y.shape[1]

# shuffle and split training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=42)
n_classes = y.shape[1]

# Learn to predict each class against the other
from sklearn.multiclass import OneVsRestClassifier
classifier = OneVsRestClassifier(svm.SVC(kernel='linear', probability=True, random_state = random_state))
y_score = classifier.fit(X_train, y_train).decision_function(X_test)

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])
    
# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

plt.figure()
lw = 2
plt.plot(fpr[0], tpr[0], color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc[0])
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

# Compute macro-average ROC curve and ROC area

# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

# Plot all ROC curves
plt.figure()
plt.plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["micro"]),
         color='deeppink', linestyle=':', linewidth=4)

plt.plot(fpr["macro"], tpr["macro"],
         label='macro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["macro"]),
         color='navy', linestyle=':', linewidth=4)

colors = cycle(['aqua', 'darkorange'])
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Some extension of Receiver operating characteristic to multi-class')
plt.legend(loc="lower right")
plt.show()

from sklearn.metrics import roc_auc_score
print('ROC AUC Score: ', roc_auc_score(y_test, y_score))

# Receiver Operating Characteristic (ROC) with cross validation
from sklearn.model_selection import StratifiedKFold

# Run classifier with cross-validation
cv = StratifiedKFold(n_splits=6)
classifier = svm.SVC(kernel='linear', probability=True, random_state=random_state)

mean_tpr = 0.0
mean_fpr = np.linspace(0, 1, 982) # (change based on size of data being run)

y = data_manipulation[['aPTT_CAT']].copy()
y = label_binarize(y, classes=['SUB-TH', 'TH', 'SUPRA-TH'])
#y = y.applymap(int) #change: make y values a one row matrix of ints
y_list = y['aPTT_CAT'].tolist()
y_new = []
for i in range(len(y_list)):
    y_new.append(y_list[i])
y = np.array(y_new)

X = X_cat_day.copy()
X = X.applymap(int).as_matrix() #change: make X values a matrix of ints
n_samples, n_features = X.shape 

colors = cycle(['cyan', 'indigo', 'seagreen', 'yellow', 'blue', 'darkorange'])
lw = 2

i = 0
for (train, test), color in zip(cv.split(X, y), colors):
    probas_ = classifier.fit(X[train], y[train]).predict_proba(X[test])
    # Compute ROC curve and area the curve
    fpr, tpr, thresholds = roc_curve(y[test], probas_[:, 1])
    mean_tpr += interp(mean_fpr, fpr, tpr)
    mean_tpr[0] = 0.0
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, lw=lw, color=color,
             label='ROC fold %d (area = %0.2f)' % (i, roc_auc))

    i += 1

plt.plot([0, 1], [0, 1], linestyle='--', lw=lw, color='k', 
         label='Luck')

mean_tpr /= cv.get_n_splits(X, y)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
plt.plot(mean_fpr, mean_tpr, color='g', linestyle='--',
         label='Mean ROC (area = %0.2f)' % mean_auc, lw=lw)

plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

